<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 4 / SPRINT 1 / NOTE 3*

---
# N413. 신경망 구현을 위한 프레임워크 - Tensorflow, Keras


## Warm up

다음 영상을 시청해 주세요 :)
- [하이퍼파라미터 튜닝이 뭐지?](https://www.youtube.com/watch?v=wKkcBPp3F1Y)
- [파라미터? 하이퍼파라미터?](https://youtu.be/Kh06wgGbi78?t=12)
-[학습 규제 방식에 대한 설명 강의](https://youtu.be/_sz3KTyB9Lk?t=1005)
  * [L1/L2-regularization](https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c)


# 지난시간 복습
- 신경망의 동작원리 (Note1)
  * 데이터 전처리 및 입력
  * 모델 제작 및 가중치 초기화
  * 모델에 데이터를 넣고 출력값을 얻음
  * 출력값과 레이블(정답지)과 비교 후 Loss 계산
  * Loss를 반영하여 가중치 업데이트 -> 역전파(BackPropagation) + 경사하강법(Gradient Descent)
- 역전파 원리 및 실습
  * Loss function의 계산방식
  * Stochastic Gradient Descent 방법
  * 경사하강법의 변형들(Adam)
  * 2x2x2 neural network의 역전파 수학식
- Fashion MNIST 실습

<br>

---
<br>

# 🏆 학습목표
* <a href="#p1">Part 1</a>: 모델 아키텍쳐를 어떻게 선택하는 지 배우게 됩니다.
* <a href="#p3">Part 2</a>: 가중치의 규제(Regularization) 전략을 배웁니다. 
* <a href="#p2">Part 3</a>: 다양한 활성함수를 사용함에 발생하는 trade-off에 대해서 논의해볼 수 있어야 합니다. 




## 딥러닝 라이브러리를 사용해봅시다!

지난 이틀간의 목표는 신경망의 배경, 기초, 용어, 네트워크 구조, 전파/역전파, 오류/비용 함수, 에폭(Epoch), 그리고 경사하강법 등을 숙지하는 것이었죠. Perceptrons(단일 노드 신경망)와 Feed-Forward Neural Networks라고도 알려진 Multi-Layer Perceptrons를 포함하여 간단한 신경망을 손으로 코딩하도록 요구하여 함으로써 신경망에 익숙해지기 위해 노력해왔죠. 

수작업으로 한땀 한땀 신경망을 만드는 것은 우리의 시간을 사용하는 최선의 방법은 아닐 것이라는 것을 아실 것이고, 이제는 보다 편하게 모듈화된 자료들을 하나씩 배워볼 수 있습니다. 실무에서 사용할 예측 모델을 만들기 위해 강력한 라이브러리를 사용하기 시작할 것입니다. Let's Go!



> 딥러닝 연구자들의 일부는 신경망을 위한 아키텍쳐(구조)을 선택하는 것은 과학이라기 보다는 예술에 가깝다고 말합니다. 

> 한편, 노가다라고 말하기도합니다. 0.1%를 올리기 위해서 수많은 작업들이 진행되기도 하니까요.

> 용도에 맞는 구조를 선택하는 가장 좋은 방법은 연구와 실험을 통해서 발견할 수 있기 때문입니다.

In [ ]:
# 파일 선택을 통해 예제 데이터를 내 컴퓨터에서 불러오는 코드를 포함(주석)
# 강의 목적상 내 데이터를 대신하여 서버에서 불러오도록 하겠습니다. 직접 가지고 있는 데이터를 사용하기 위해서는 주석처리된 files.upload()를 이용하시면 됩니다.
from google.colab import files
#uploaded = files.upload() # 파일을 불러올 수 있는 코드

# 폐암 수술 환자의 특정기간 생존 데이터
# 속성(정보)은 종양의 유형, 폐활량, 호흡곤란 여부, 기침, 흡연, 천식여부 등의 17가지 환자 상태. 수술 후 생존(1), 사망(0) 
my_data = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/everydeep/ThoraricSurgery.csv"

# 딥러닝을 구동하는 데 필요한 케라스 함수를 불러옵니다.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.layers as Layer

# 케라스 외의 필요한 라이브러리를 불러옵니다.
import numpy as np
import pandas as pd
import tensorflow as tf

# 실행할 때마다 같은 결과를 출력하기 랜덤함수를 고정하는 부분입니다.
# 랜덤함수의 Seed를 고정하게 되면 랜덤함수가 항상 일정하게 나옵니다. 
np.random.seed(3)
tf.random.set_seed(3)

# 불러온 데이터를 적용합니다.
# pandas외에도 읽을 수 있는 방법이 있습니다. 편하신 방법을 사용하시면 됩니다.
Data_set = np.loadtxt(my_data, delimiter=",") 

In [ ]:
Data_set = pd.read_csv(my_data, header=None) 
Data_set.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000
mean,235.500000,3.095745,3.281638,4.568702,0.780851,0.065957,0.144681,0.065957,0.687234,0.165957,11.736170,0.074468,0.004255,0.017021,0.821277,0.004255,62.534043,0.148936
std,135.821574,0.722309,0.871395,11.767857,0.535375,0.248472,0.352154,0.248472,0.464114,0.372439,0.702243,0.262811,0.065163,0.129488,0.383529,0.065163,8.706902,0.356405
min,1.000000,1.000000,1.440000,0.960000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000
25%,118.250000,3.000000,2.600000,1.960000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000,1.000000,0.000000,57.000000,0.000000
50%,235.500000,3.000000,3.160000,2.400000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,12.000000,0.000000,0.000000,0.000000,1.000000,0.000000,62.000000,0.000000
75%,352.750000,3.000000,3.807500,3.080000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,12.000000,0.000000,0.000000,0.000000,1.000000,0.000000,69.000000,0.000000
max,470.000000,8.000000,6.300000,86.300000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,14.000000,1.000000,1.000000,1.000000,1.000000,1.000000,87.000000,1.000000


In [ ]:
Data_set = np.loadtxt(my_data, delimiter=",") 
# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장합니다.
X = Data_set[:,0:17]
Y = Data_set[:,17]

# 딥러닝 구조를 결정합니다(모델을 설정하고 실행하는 부분입니다).
model = Sequential([
    Dense(30, activation='relu'),
    Layer.Dropout(0.5),
    Dense(30, activation='relu'),
    Dense(1, activation='sigmoid') # 분류할 방법에 따라 개수를 조정해야 합니다. 
])
# 딥러닝을 실행합니다.
model.compile(loss='mean_squared_logarithmic_error', optimizer='adam', metrics=['accuracy']) # mean_squared_error # binary_crossentropy # mean_absolute_error # poisson
history = model.fit(X, Y, epochs=30, batch_size=30)

Epoch 1/30
16/16 [==============================] - 1s 2ms/step - loss: 0.1923 - accuracy: 0.5833
Epoch 2/30
16/16 [==============================] - 0s 1ms/step - loss: 0.0986 - accuracy: 0.7804
Epoch 3/30
16/16 [==============================] - 0s 2ms/step - loss: 0.0887 - accuracy: 0.8106
Epoch 4/30
16/16 [==============================] - 0s 2ms/step - loss: 0.0765 - accuracy: 0.8378
Epoch 5/30
16/16 [==============================] - 0s 2ms/step - loss: 0.0742 - accuracy: 0.8453
Epoch 6/30
16/16 [==============================] - 0s 2ms/step - loss: 0.0683 - accuracy: 0.8559
Epoch 7/30
16/16 [==============================] - 0s 2ms/step - loss: 0.0735 - accuracy: 0.8452
Epoch 8/30
16/16 [==============================] - 0s 2ms/step - loss: 0.0753 - accuracy: 0.8432
Epoch 9/30
16/16 [==============================] - 0s 2ms/step - loss: 0.0669 - accuracy: 0.8616
Epoch 10/30
16/16 [==============================] - 0s 2ms/step - loss: 0.0632 - accuracy: 0.8710
Epoch 11/30
16/16 [

### (복습) 오차함수

평균제곱계열
- (복습) mean_squared_error (MSE) = $\frac{1}{n}\sum_{i=1}^{n}(y_{i} - \hat{y_{i}})^{2}$
- (복습) RMSE (Root Mean Squared Error) = 
$\sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_{i} - \hat{y_{i}})^{2}}$
- (복습) mean_absolute_error (MAE) = $\frac{1}{n}\sum_{i=1}^{n}\left | y_{i} - \hat{y_{i}} \right |$
- (복습) R-Squared (coefficient of determination) = $1 - \frac{\sum_{i=1}^{n}(y_{i} - \hat{y_{i}})^{2}}{\sum_{i=1}^{n}(y_{i} - \bar{y_{i}})^{2}} = 1 - \frac{SSE}{SST} = \frac {SSR}{SST}$
  -  SSE, SST, SSR: Sum of Squared `Error`, `Total`, `Regression`($\sum_{i=1}^{n}(\hat{y_{i}} - \bar{y_{i}})^{2}$)
- mean_absolute_percentage_error = $ \frac {1}{n}\sum _{i=1}^{n}\left|{\frac {y_{t}-\hat{y_{i}}}{y_{i}}}\right| $
- mean_squared_logarithmic_error = $\frac{1}{n} \sum_{i=1}^n (\log(\hat{y_i} + 1) - \log(y_i+1))^2 $



엔트로피계열
- binary_crossentropy = $ -\sum_{c=1}^{C} q(y_c) log(q(y_c)), \hspace{2em} q(y_c) \in (1, -1)$
- categorical_crossentropy = $ -\sum_{c=1}^{C} q(y_c)log(q(y_c)) $


[기타 다른 계열](https://keras.io/api/losses/)

## 학습 규제 전략 (Regularization Strategies)

### Overfitting 극복을 위한 노력

Neural Networks는 매개변수가 아주 많은 모델이어서, Section 2에서 공부했던 것처럼 훈련 데이터에 쉽게 과대적합(overfit) 오버핏될 수 있다. 이 문제를 해결하는 가장 중요한 방법은 가중치 규제 전략이다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/Regularization.svg/1920px-Regularization.svg.png" width = 600 />


### 일반적인 규제방법
우리가 간단히 다루는 신경망에는 네 가지 일반적인 가중치 규제 방법이 있다. 이러한 구성 요소를 적용하는 방법:

1. 항상 EarlyStopping을 사용한다. 이 전략은 당신의 가중치의 최고 유용성 시점을 훨씬 지나서 더 업데이트되는 것을 막을 것이다.
<img src=https://miro.medium.com/max/474/1*wZg_RQHPRtn62dDp2Ez86A.jpeg>

2. 가중치 감소(Weight Decay)

<img src=https://camo.githubusercontent.com/7d9e05f214d77fcb9ce7b13d56448b51ed4169ed63495778678c201da61f4436/68747470733a2f2f692e737461636b2e696d6775722e636f6d2f6f564a44422e706e67>

3. Weight Constraint
Weight Decusion and Weight Restriction은 유사한 목적을 위하여 가중치를 제거하거나 값을 규제하여 매개변수를 과도하게 적합시키는 것을 방지하는 역할이다. 같은 목적의 다른 방법이기 때문에 이들을  굳이 같이 적용하지 않아도 된다. 

4. Dropout 사용

<img src=https://miro.medium.com/max/981/1*EinUlWw1n8vbcLyT0zx4gw.png>

신경망의 각 레이어 노드에서 학습할 때 마다 [일부 노드를 사용하지 않고 학습을 진행](https://m.blog.naver.com/PostView.nhn?blogId=isu112600&logNo=221578533182&proxyReferer=https:%2F%2Fwww.google.com%2F)하는 방법이다. 노드들 같의 연결(가중치) 자체를 사용하지 않도록 만들면서 하나의 모델을 여러가지 방법으로 학습을 하도록 하면서 overfitting을 방지하는 기술이다.


In [ ]:
# Tensorflow에서 데이터를 가져와 규제 하는 코드

from tensorflow.keras.datasets import fashion_mnist

# 데이터 불러오기
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
print(X_train.shape, X_test.shape)

# 데이터를 정규화 합니다
X_train = X_train / 255.
X_test = X_test /255.

# 클래스를 확인합니다.
np.unique(y_train)

4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28) (10000, 28, 28)


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
# 기본적인 신경망을 만드는 코드

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import keras, os

# 모델 구성을 확인합니다.
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(10, activation='softmax')
])
# 업데이트 방식을 설정합니다.
model.compile(optimizer='adam'
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
model.summary()
# 총 7850 parameters (10 bias)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 학습을 위한 코드

# 변수 설정을 따로 하는 방법을 적용하기 위한 코드입니다. 
batch_size = 30
epochs_max = 1

# 학습시킨 데이터를 저장시키기 위한 코드입니다. 
checkpoint_filepath = "FMbest.hdf5"

# overfitting을 방지하기 위해서 학습 중 early stop을 수행하기 위한 코드입니다.
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

# Validation Set을 기준으로 가장 최적의 모델을 찾기 위한 코드입니다.
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)

# 모델 학습 코드 + early stop + Best model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs_max, verbose=1, 
          validation_data=(X_test,y_test), 
          callbacks=[early_stop, save_best])

2000/2000 [==============================] - 4s 2ms/step - loss: 0.7801 - accuracy: 0.7314 - val_loss: 0.5107 - val_accuracy: 0.8221

Epoch 00001: val_loss improved from inf to 0.51069, saving model to FMbest.hdf5


In [ ]:
# 학습된 모델을 이용하여 테스트하는 코드

model.predict(X_test[0:1])
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.5107 - accuracy: 0.8221


In [ ]:
!ls 

ds-lecture-data.s3.ap-northeast-2.amazonaws.com  FMbest.hdf5  sample_data


In [ ]:
# 체크포인트에 저장된 가중치들을 불러들이는 코드

model.load_weights(checkpoint_filepath)

In [ ]:
# best model을 이용한 테스트 데이터 예측 정확도 재확인 코드

model.predict(X_test[0:1])
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

313/313 - 0s - loss: 0.5107 - accuracy: 0.8221


# Keras에서 규제방법을 구현해보자
```
Dense(64, input_dim=64,
            kernel_regularizer=regularizers.l2(0.01),
            activity_regularizer=regularizers.l1(0.01))
```

### Weight Decay

$L(\theta_w) = {1 \over 2} \Sigma_i (output_i - target_i)^2 + \lambda |\theta_w|$ 

$L(\theta_w) = {1 \over 2} \Sigma_i (output_i - target_i)^2 + \lambda ||\theta_w||_2$ 

이번에는 학습과정에서 Overfitting을 방지하기 위한 기술, Regularization의 한 종류인 Weight Decay를 수행해보겠습니다. 말 그대로 가중치를 감소시키는 기술입니다. 아래 그림을 다시한번 보면, 굽이치는 그래프를 나타내려면 큰 가중치가 필요합니다. 애초에 큰 가중치를 갖지 못하게 만들면, 다음과 같이 과대적합이 될 수가 없도록 만드는 기술입니다.

조금더 자세히 설명해보면, 위 식에서 “연산된 결과인 output”과 “Target”의 차이 제곱식 말고, 뒤에 $\lambda ||$로 구성된 term이 하나 생겼습니다. 달라진 점은 이곳 하나죠. Loss function은 다른말로 objective function이라고도 하는데요, 우리의 목적을 달성하는 기준 함수가 되기 때문입니다. 여기에 가중치가 너무 커지게되는 overfit를 방지하고자, w 크기 자체가 error로 반영이 되면, w의 크기가 커지지 않으면서도 output과 target의 차이가 줄어들게 하겠다는 의미로 받아들이시면 됩니다. 

<img src="https://miro.medium.com/max/1400/0*CmDTGlQyibHUORQ0.png"/>


In [ ]:
# Weight Decay를 전체적으로 반영한 예시 코드
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras import regularizers

# 모델 구성을 확인합니다.
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, 
            kernel_regularizer=regularizers.l2(0.01),    # L2 norm regularization
            activity_regularizer=regularizers.l1(0.01)), # L1 norm regularization    
    Dense(10, activation='softmax')
])
# 업데이트 방식을 설정합니다.
model.compile(optimizer='adam'
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
model.summary()

model.fit(X_train, y_train, batch_size=30, epochs=1, verbose=1, 
          validation_data=(X_test,y_test))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
2000/2000 [==============================] - 4s 2ms/step - loss: 1.3607 - accuracy: 0.7677 - val_loss: 0.8033 - val_accuracy: 0.8060


[Overfitting](https://towardsdatascience.com/over-fitting-and-regularization-64d16100f45c)

[L2 regularization](https://developers.google.com/machine-learning/crash-course/regularization-for-simplicity/l2-regularization)

[L1 vs L2 한국어 설명](https://light-tree.tistory.com/125)

### Constraints

[참고자료](https://keras.io/api/layers/constraints/)

Weight Decay를 통해서 Weight의 학습 반경을 변경시켰다면, 이번에는 물리적으로 Weight의 크기를 제한하는 방법입니다. Weight자체를 함수를 이용하여 더 큰 경우는 임의의 값으로 변경해버리는 기술을 사용하게 됩니다. 그러면 더이상 커질 수가 없겠죠? 

In [ ]:
# 모델 구성을 확인합니다.
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, 
            kernel_regularizer=regularizers.l2(0.01),
            activity_regularizer=regularizers.l1(0.01),
            kernel_constraint=MaxNorm(2.)),             ## add constraints
    Dense(10, activation='softmax')
])
# 업데이트 방식을 설정합니다.
model.compile(optimizer='adam'
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
model.summary()


model.fit(X_train, y_train, batch_size=30, epochs=1, verbose=1, 
          validation_data=(X_test,y_test))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_7 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
2000/2000 [==============================] - 5s 2ms/step - loss: 1.3686 - accuracy: 0.7639 - val_loss: 0.7866 - val_accuracy: 0.8135


### Dropout

Dropout의 경우는 위의 경우와는 다른 방식으로 overfitting을 방지합니다. 
모델 자체에 Layer를 추가하는 방식으로 진행이 되는데요, 이는 확률적으로 노드 연결을 강제로 끊어주는 역할을 합니다. 보톡스를 맞으면 근육을 쓰지 못하게 해서 주름이 생기는 것을 막아버리는데요, 뉴럴넷의 보톡스와 같은 존재라고 할 수 있습니다.  단, 임시로 차단을 하고, 그 연결이 없이 결과를 예측하도록 만들고, 해당 뉴런없이 학습을 진행하기 때문에 과적합을 어느정도 차단할 수 있습니다. 

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras import regularizers
# 모델 구성을 확인합니다.
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, 
            kernel_regularizer=regularizers.l2(0.01),
            activity_regularizer=regularizers.l1(0.01),
            kernel_constraint=MaxNorm(2.)),             
    Dropout(0.5)       ,                                   ## add dropout
    Dense(10, activation='softmax')
])
# 업데이트 방식을 설정합니다.
model.compile(optimizer='adam'
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
model.summary()


model.fit(X_train, y_train, batch_size=30, epochs=1, verbose=1, 
          validation_data=(X_test,y_test))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
2000/2000 [==============================] - 5s 2ms/step - loss: 1.5921 - accuracy: 0.7224 - val_loss: 0.8783 - val_accuracy: 0.8052


### 조금 더 나은 모델학습을 위하여!

이번에는 조금 다른 기술들에 대해서 배워보겠습니다.  

무엇인가 추가하는 방법이 아니라 기존의 방법을 어떻게 활용할 지에 대한 고민입니다. 위의 논문은 실무에 들어가시게 되면 꼭 한번 읽어보세요.

CNN에 적용되는 기술로 소개되었지만, 최근에는 그 경계가 사라졌고, 지금 배워도 유용한 기술이기 때문에 미리 배워보겠습니다.

학습을 어떻게 효율적으로 할 것인가? 

[참고논문](https://https://openaccess.thecvf.com/content_CVPR_2019/papers/He_Bag_of_Tricks_for_Image_Classification_with_Convolutional_Neural_Networks_CVPR_2019_paper.pdf)
Bag of Tricks for Image Classification with Convolutional Neural Networks

#### Learning rate decay

decay 전략은 다음처럼 활용할 수 있습니다.

```
tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)
```



In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1 = 0.89)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
model.summary()


model.fit(X_train, y_train, batch_size=30, epochs=1, verbose=1, 
          validation_data=(X_test,y_test))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
2000/2000 [==============================] - 5s 2ms/step - loss: 0.9318 - accuracy: 0.7889 - val_loss: 0.8332 - val_accuracy: 0.8121


+ 이제 신경망에 조금 익숙해졌다면, 파라미터 개수도 유심히 보면 좋습니다. 

   784 x 64 + 64 = 50240

#### learning rate 스케쥴링



```
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
```



In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
model.summary()


model.fit(X_train, y_train, batch_size=30, epochs=1, verbose=1, 
          validation_data=(X_test,y_test))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
2000/2000 [==============================] - 5s 2ms/step - loss: 2.2620 - accuracy: 0.6633 - val_loss: 1.5827 - val_accuracy: 0.7125


In [ ]:
def decayed_learning_rate(step):
  step = min(step, decay_steps)
  cosine_decay = 0.5 * (1 + cos(pi * step / decay_steps))
  decayed = (1 - alpha) * cosine_decay + alpha
  return initial_learning_rate * decayed

first_decay_steps = 1000
initial_learning_rate = 0.01
lr_decayed_fn = (
  tf.keras.experimental.CosineDecayRestarts(
      initial_learning_rate,
      first_decay_steps))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
model.summary()


model.fit(X_train, y_train, batch_size=30, epochs=1, verbose=1, 
          validation_data=(X_test,y_test))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
2000/2000 [==============================] - 5s 2ms/step - loss: 1.9928 - accuracy: 0.6509 - val_loss: 1.8841 - val_accuracy: 0.7224


# + Activation Functions (Plus 학습)

### Tanh Function

![Tanh Function](http://mathworld.wolfram.com/images/interactive/TanhReal.gif)

만약 sigmoid 함수가 0에서 멀어질 때 바로 그렇게 평평해지지 않고 중간에 조금 더 뾰족해진다면? 그것은 기본적으로 Tanh 기능이다. Tanh 함수는 실제로 y 치수에서 sigmoid 함수를 2로 스케일링하고 모든 값에서 1을 빼면 생성될 수 있다. 기본적으로 sigmoid와 동일한 성질을 가지며, 여전히 우리가 0에서 멀어질수록 평평한 구배 감소에 어려움을 겪고 있지만, 그 파생상품은 0 주위에 더 높아져 가중치가 극단으로 조금 더 빠르게 이동한다.

### Leaky ReLU

<img src="https://cdn-images-1.medium.com/max/1600/1*ypsvQH7kvtI2BhzR2eT_Sw.png" width=600/>

ReLU가 제일 좋다고만 들었는데, 인성에 문제가 있어? 보통 그래프의 왼쪽 절반(음수)의 함수는 뉴런이 활성화되지 않도록 하는 것을 알고 있죠.  가중치로 초기화된 뉴런의 경우, 우리의 구배는 뉴런의 가중치를 업데이트하지 않을 것이며, 이것은 결코 발화하지 않고 가중치이 업데이트하지 않는 죽은 뉴런, 쓸데없이 메모리를 차지하는 뉴런으로 될 수 있음을 보여준다. 우리는 아마도 초기 가중치가 안 좋게 생성되는 경우를 대비해서 조금이라도 발화하지 않는 뉴런의 가중치를 업데이트하고 미래에 다시 켤 수 있는 기회를 주고 싶을 것이다.

Leaky ReLU는 정확히 그것을 해결합니다! 파생 기능 왼쪽(음수)에서 0의 경사를 피함으로 해결합니다. 이는 '죽은' 뉴런도 충분한 반복에 의해 재생될 가능성이 있다는 것을 의미한다. 일부 규격에서는 누출되는 좌측의 기울기를 모델의 하이퍼 파라미터로 실험할 수도 있다!

### Softmax Function

![Softmax Function](https://cdn-images-1.medium.com/max/800/1*670CdxchunD-yAuUWdI7Bw.png)

sigmoid 함수와 유사하지만 다중 클래스 분류 문제에 더 유용하다. 소프트맥스 함수는 모든 입력 집합을 취하여 최대 1까지 합한 확률로 변환할 수 있다. 이것은 우리가 어떤 출력물 목록을 던질 수 있다는 것을 의미하며, 그것은 확률로 변환할 것이고, 이것은 다중 클래스 분류 문제에 매우 유용하다. 예를 들어 MNIST처럼...

# Review
* Keras의 예제 튜토리얼을 통해서 나 혼자 노트를 만들어 볼 수 있는 지 확인해본다.
* 학습 규제 전략에 대해서 개념을 설명할 수 있다. 
  - Weight Decay
  - Weight constriant
  - Drop-Out
  - Learning rate 
* 새롭게 배운 Activation function에 대해서 설명할 수 있다.
  - hyperbolic tangent (Tanh)
  - Leaky ReLU
  - Softmax

## 참고자료
- [OpenCV를 이용한 MNIST 인식 모델 만들어보기](https://www.youtube.com/watch?v=TV3oplqa5VA) + 더보기 (코드 링크 있음)
- [Tensorflow를 이용한 CNN 실습영상](https://www.youtube.com/watch?v=pZGvMhhawy8) + 더보기 (코드 링크 있음)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip freeze > "drive/MyDrive/Colab Notebooks/section4/N423_requirements.txt"

In [ ]:
!ls drive/MyDrive/Colab\ Notebooks/section4